In [15]:
import pandas as pd
import numpy as np
import re

# Have a look at the two datasets 

In this stage, we use some predictors from sweep0-3 and sweep5.

In [16]:
sweep5 = pd.read_csv("ncds_sweep5/tab/ncds5cmi.tab",delimiter="\t",low_memory=False)
sweep5.head(10)

,ncdsid,person,n622_5,n5region,n5gor,resp5cmi,resp5wyt,resp5cl,n500518,n500520,...,child,marchild,hqual33,hqual23,verbal,constrct,keyboard,caring,organise,tenure91
0,N10001N,1,2,6,9,1,1,1,0,2,...,1,2,2,2,3.5,3,3,2.66666666666667,3.4,2
1,N10002P,1,1,5,8,1,1,1,0,1,...,2,5,4,4,4,4,4,3.66666666666667,3.8,2
2,N10004R,1,2,1,2,1,1,1,0,5,...,1,2,2,2,4,3.33333333333333,1.5,3.33333333333333,3.4,3
3,N10007U,1,2,6,9,1,1,1,0,1,...,2,6,3,3,3,3,3.5,3,2.8,2
4,N10009W,1,1,5,7,1,1,1,0,1,...,2,7,2,-1,4,4,3,3.66666666666667,2.2,99
5,N10011Q,1,1,7,5,1,1,1,0,1,...,1,3,3,3,3.5,3.33333333333333,4,3.33333333333333,3.4,3
6,N10012R,1,1,6,9,1,1,1,0,3,...,1,1,4,0,3.5,4,1,3.33333333333333,,4
7,N10013S,1,1,3,4,1,1,1,0,1,...,1,1,1,1,3.5,3.33333333333333,4,4,3.6,2
8,N10015U,1,1,1,1,1,0,0,0,1,...,1,1,1,1,,,,,,2
9,N10016V,1,2,4,6,1,1,1,0,2,...,1,2,1,1,3.5,3.33333333333333,3,3.33333333333333,3.4,2


In [17]:
sweep5.shape

(11469, 3189)

In [18]:
sweep0_3 = pd.read_csv("ncds_sweep0-3/tab/ncds0123.tab",delimiter="\t",low_memory=False)
sweep0_3.head(10)

,ncdsid,n622,n0region,n1region,n2region,n3region,n553,n545,n520,n490,...,n1849,dvht07,dvht11,dvht16,dvrwt07,dvrwt11,dvrwt16,dvwt07,dvwt11,dvwt16
0,N10001N,2,9,9,9,9,23,4,2,12,...,-1,1.21899986267032,1.47299957275415,1.59999942779607,110.347991943347,98.1929931640604,105.055999755876,25.8549957275385,37.6489868164152,56.0199890136717
1,N10002P,1,9,8,8,8,34,4,5,1,...,-1,1.34599971771224,-1,-1,90.865997314449,-1,-1,26.3089904785155,-1,-1
2,N10003Q,1,4,4,4,4,34,4,10,1,...,-1,1.32099914550831,1.49899959564243,1.87999916076665,87.9599914550983,96.4049987792867,89.382995605487,24.4939880371087,38.1019897460905,66.6799926757659
3,N10004R,2,1,1,1,1,26,4,11,1,...,-1,1.29499912262003,1.51099967956562,1.62999916076665,105.16198730471,111.588989257796,132.054992675766,28.122985839843,45.8139953613169,72.7999877929584
4,N10005S,2,10,10,10,10,25,4,1,3,...,-1,1.34599971771224,-1,-1,129.382995605487,-1,-1,37.6489868164152,-1,-1
5,N10006T,1,7,4,7,7,23,4,9,1,...,,1.21899986267032,1.40999984741198,-1,96.2529907226337,94.2619934081847,-1,22.6799926757811,31.7519989013717,-1
6,N10007U,2,-2,-1,-1,2,,,,,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,N10008V,1,10,-1,10,10,21,4,10,3,...,-1,-1,1.26999950408932,1.59999942779607,-1,104.393997192364,91.8689880371285,-1,26.761993408203,45.8099975585734
8,N10009W,1,-2,7,7,-1,,,,,...,,1.19399929046639,1.37799930572474,-1,-1,82.9279937743942,-1,-1,26.3089904785155,-1
9,N10010P,2,7,7,7,7,26,4,1,1,...,-1,1.26999950408932,1.57499980926536,-1,-1,118.603988647462,-1,-1,54.4319915771376,-1


In [19]:
sweep0_3.shape

(18558, 1765)

# Sweep 0_3 wrangling

Test a method to filter out the variables.

In [7]:
a = pd.DataFrame({"1P abc": [1,2,3], "1PM abc":[4,5,6]})
a

,1P abc,1PM abc
0,1,4
1,2,5
2,3,6


In [8]:
#check whether the variable name contains 1P, if not, drop the column
def drop_column(df, names):
    
    columns = df.columns.values
    for column in columns:
        if column.split()[0] in names:
            continue
        else:
            df.drop(column,1,inplace = True)  
    return df

drop_column(a,["1PM"])

,1PM abc
0,4
1,5
2,6


# Sweep 5 wrangling

In the following steps, I will filter out the health variables from sweep 5

In [9]:
var = pd.read_csv("ncds_sweep5/tab/health_var.txt", delimiter="\t",names=["label","others"])
var.head()

,label,others
0,N504147,CMI:83 F34 Any accident/assault since March 1981
1,N504148,CMI:83 F35 No. of accidents/assaults since Mar...
2,N504150,CMI:83 F36b) ACCIDENT/ASSAULT 1: Age occurred
3,N504152,CMI:83 F36c) ACCIDENT/ASSAULT 1: Incident type
4,N504153,CMI:83 F36d) ACCIDENT/ASSAULT 1: Admitted/outp...


In [10]:
#get list of health variables and predictors
var_health = [x.lower() for x in var["label"].values]
var_health.insert(0, "ncdsid")
print var_health[:3]

['ncdsid', 'n504147', 'n504148']


Nice!! Then I can get a new dataframe only with these variables. For example:

In [11]:
sweep5[var_health].head()

,ncdsid,n504147,n504148,n504150,n504152,n504153,n504154,n504156,n504157,n504158,...,n504475,n504476,n504513,n504514,n504515,n504516,n504517,n504518,n504519,n504520
0,N10001N,2,,,,,,,,,...,,,1,2,,,,,,
1,N10002P,2,,,,,,,,,...,,,1,2,,,,,,
2,N10004R,2,,,,,,,,,...,,,1,2,,,,,,
3,N10007U,2,,,,,,,,,...,,,1,2,,,,,,
4,N10009W,2,,,,,,,,,...,,,1,2,,,,,,


# Merge them and set x and y

Now we can work on the whole dataset! Merge sweep 0-3 and sweep 5!

In [20]:
def get_labels(path,flag):

    # open explain file
    explain = open(path, 'r')

    # read the whole file as a single giant string
    explain_text = explain.read()

    # locate variable target string in the whole file
    target_string_var = 'Variable label ='
    target_loc_var = [m.end() for m in re.finditer(target_string_var, explain_text)]
    var_name_list = []
    # locate type target string in the whole file
    target_string_type = 'the SPSS measurement level is'
    target_loc_type = [m.end() for m in re.finditer(target_string_type, explain_text)]
    type_list = ["Categorical" for i in target_loc_var]

    #Find Numeric or Categorical
    for index, loc in enumerate(target_loc_type):
        # get feature character
        char = explain_text[loc+10]
        if char == "S" or char == "s":
            # it is SPSS data type ORDINAL or NOMINAL
            # it is categorical data
            type_list[index] = "Numeric"

            
    #Find variable label
    if flag == 0:
        for loc in target_loc_var:
            start_point = loc+11
            end_point = explain_text.find('\\\n', loc)
            var_name_list.append(explain_text[start_point:end_point])
    elif flag ==1:
        for loc in target_loc_var:
            start_point = loc+8
            end_point = explain_text.find('\par', loc)
            var_name_list.append(explain_text[start_point:end_point])
        
    explain.close()   
    return var_name_list, type_list

In [21]:
# file path of explain file
sweep5_path = 'ncds_sweep5/mrdoc/allissue/ncds5cmi_ukda_data_dictionary.rtf'
sweep0_3_path = 'ncds_sweep0-3/mrdoc/allissue/ncds0123_ukda_data_dictionary.rtf'
sweep0_3_labels, sweep0_3_type = get_labels(sweep0_3_path,1)
sweep5_labels, sweep5_type  = get_labels(sweep5_path,0)
print len(sweep0_3_labels), len(sweep5_labels)

1765 3189


Finally! We have everything we need to merge the two datasets!

In [15]:
#filter sweep0-3
sweep0_3.columns = sweep0_3_labels
sweep0_3.head()

,ncdsid serial number,0-3D Sex of child,Region at PMS (1958) - Birth,Region at NCDS1 (1965) - 7 years,Region at NCDS2 (1969) - 11 years,Region at NCDS3 (1974) - 16 years,"0 Mother's age last birthday,in years",0 Mother's present marital status,0 Interval between marriage & 1st birth,0 Socio-economic group mother's husband (GRO 1951),...,3D Losses at NCDS3,1D Height in metres at 7 years,2D Height in metres at 11 years,3D Height in metres at 16 years,1D Relative weight at 7 years,2D Relative weight at 11 years,3D Relative weight at 16 years,1D Weight in Kg. at 7 years,2D Weight in Kg. at 11 years,3D Weight in Kg. at 16 years
0,N10001N,2,9,9,9,9,23,4,2,12,...,-1,1.21899986267032,1.47299957275415,1.59999942779607,110.347991943347,98.1929931640604,105.055999755876,25.8549957275385,37.6489868164152,56.0199890136717
1,N10002P,1,9,8,8,8,34,4,5,1,...,-1,1.34599971771224,-1,-1,90.865997314449,-1,-1,26.3089904785155,-1,-1
2,N10003Q,1,4,4,4,4,34,4,10,1,...,-1,1.32099914550831,1.49899959564243,1.87999916076665,87.9599914550983,96.4049987792867,89.382995605487,24.4939880371087,38.1019897460905,66.6799926757659
3,N10004R,2,1,1,1,1,26,4,11,1,...,-1,1.29499912262003,1.51099967956562,1.62999916076665,105.16198730471,111.588989257796,132.054992675766,28.122985839843,45.8139953613169,72.7999877929584
4,N10005S,2,10,10,10,10,25,4,1,3,...,-1,1.34599971771224,-1,-1,129.382995605487,-1,-1,37.6489868164152,-1,-1


In [16]:
sweep0_3_filtered = drop_column(sweep0_3, ["ncdsid","1P","1PD","1M","1D","2M","2P","3P","3M"])
sweep0_3_filtered.head(10)

,ncdsid serial number,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,3M R ear-hearing loss at 8000 CPS x 5dB,3M L ear-hearing loss at 250 CPS x 5dB,3M L ear-hearing loss at 500 CPS x 5dB,3M L ear-hearing loss at 1000 CPS x 5dB,3M L ear-hearing loss at 2000 CPS x 5dB,3M L ear-hearing loss at 4000 CPS x 5dB,3M L ear-hearing loss at 8000 CPS x 5dB,1D Height in metres at 7 years,1D Relative weight at 7 years,1D Weight in Kg. at 7 years
0,N10001N,-1,1,3,1,3,5,1,2,2,...,0,0,0,0,0,0,0,1.21899986267032,110.347991943347,25.8549957275385
1,N10002P,-1,3,2,1,2,4,2,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.34599971771224,90.865997314449,26.3089904785155
2,N10003Q,-1,0,1,1,8,8,7,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.32099914550831,87.9599914550983,24.4939880371087
3,N10004R,-1,3,3,1,2,4,2,2,2,...,20,20,20,20,20,20,20,1.29499912262003,105.16198730471,28.122985839843
4,N10005S,-1,2,3,1,2,4,2,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.34599971771224,129.382995605487,37.6489868164152


**Note** Missing values are left as " ", replace them with NaN!

In [36]:
sweep0_3_filtered.replace(" ",np.nan,inplace=True)

In [37]:
sweep0_3_filtered.head(10)

,ncdsid serial number,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,3M R ear-hearing loss at 8000 CPS x 5dB,3M L ear-hearing loss at 250 CPS x 5dB,3M L ear-hearing loss at 500 CPS x 5dB,3M L ear-hearing loss at 1000 CPS x 5dB,3M L ear-hearing loss at 2000 CPS x 5dB,3M L ear-hearing loss at 4000 CPS x 5dB,3M L ear-hearing loss at 8000 CPS x 5dB,1D Height in metres at 7 years,1D Relative weight at 7 years,1D Weight in Kg. at 7 years
0,N10001N,-1,1,3,1,3,5,1,2,2,...,0,0,0,0,0,0,0,1.21899986267032,110.347991943347,25.8549957275385
1,N10002P,-1,3,2,1,2,4,2,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.34599971771224,90.865997314449,26.3089904785155
2,N10003Q,-1,0,1,1,8,8,7,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.32099914550831,87.9599914550983,24.4939880371087
3,N10004R,-1,3,3,1,2,4,2,2,2,...,20,20,20,20,20,20,20,1.29499912262003,105.16198730471,28.122985839843
4,N10005S,-1,2,3,1,2,4,2,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.34599971771224,129.382995605487,37.6489868164152
5,N10006T,-1,1,2,1,2,4,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.21899986267032,96.2529907226337,22.6799926757811
6,N10007U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,N10008V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10,30,25,20,15,15,10,-1,-1,-1
8,N10009W,-1,12,2,2,2,4,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.19399929046639,-1,-1
9,N10010P,-1,4,2,1,2,5,2,2,2,...,-1,-1,-1,-1,-1,-1,-1,1.26999950408932,-1,-1


In [17]:
#filter sweep5
sweep5.loc[-1] = sweep5_labels # adding a row
#sweep5.loc[-1] = sweep5_type
sweep5.index = sweep5.index + 1  # shifting index
sweep5 = sweep5.sort_index()
print sweep5.shape
sweep5.head()

(11470, 3189)


,ncdsid,person,n622_5,n5region,n5gor,resp5cmi,resp5wyt,resp5cl,n500518,n500520,...,child,marchild,hqual33,hqual23,verbal,constrct,keyboard,caring,organise,tenure91
0,ncdsid serial number,PERSON NUMBER,Sex of Cohort Member,Standard region at NCDS5,NCDS5 Government Office Region,Responded to Cohort Member Interview,Responded to What Do You Think Questionnaire,Responded to Your Life Since 1974 Questionnaire,Time of Interview (mins),"CMI:2,A1a) Current main economic activity",...,DV:Does respondent have child(ren)?,"DV:Sex, Partnership and Children",DV:Highest qual gained at age 33,DV:Highest qual gained at age 23,DV:Possession of Verbal Skills,DV:Possession of Construction Skills,DV:Possession of Keyboard Skills,DV:Possession of Caring Skills,DV:Possession of Organisational Skills,DV:Housing tenure in 1991
1,N10001N,1,2,6,9,1,1,1,0,2,...,1,2,2,2,3.5,3,3,2.66666666666667,3.4,2
2,N10002P,1,1,5,8,1,1,1,0,1,...,2,5,4,4,4,4,4,3.66666666666667,3.8,2
3,N10004R,1,2,1,2,1,1,1,0,5,...,1,2,2,2,4,3.33333333333333,1.5,3.33333333333333,3.4,3
4,N10007U,1,2,6,9,1,1,1,0,1,...,2,6,3,3,3,3,3.5,3,2.8,2


In [18]:
sweep5_filtered = sweep5[var_health]
sweep5_filtered.head()

,ncdsid,n504147,n504148,n504150,n504152,n504153,n504154,n504156,n504157,n504158,...,n504475,n504476,n504513,n504514,n504515,n504516,n504517,n504518,n504519,n504520
0,ncdsid serial number,CMI:83 F34 Any accident/assault since March 1981,CMI:83 F35 No. of accidents/assaults since Mar...,CMI:83 F36b) ACCIDENT/ASSAULT 1: Age occurred,CMI:83 F36c) ACCIDENT/ASSAULT 1: Incident type,CMI:83 F36d) ACCIDENT/ASSAULT 1: Admitted/outp...,CMI:83 F36b) ACCIDENT/ASSAULT 2: Age occurred,CMI:83 F36c) ACCIDENT/ASSAULT 2: Incident type,CMI:83 F36d) ACCIDENT/ASSAULT 2: Admitted/outp...,CMI:83 F36b) ACCIDENT/ASSAULT 3: Age occurred,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic of...,CMI:95 F75c) Additional traffic offence convic...,CMI:95 F75c) Additional traffic offence convic...,CMI:95 F75c) Additional traffic offence convic...,CMI:95 F75c) Additional traffic offence convic...,CMI:95 F75c) Additional traffic offence convic...,CMI:95 F75c) Additional traffic offence convic...
1,N10001N,2,,,,,,,,,...,,,1,2,,,,,,
2,N10002P,2,,,,,,,,,...,,,1,2,,,,,,
3,N10004R,2,,,,,,,,,...,,,1,2,,,,,,
4,N10007U,2,,,,,,,,,...,,,1,2,,,,,,


In [19]:
sweep5_filtered = sweep5_filtered.rename(columns = sweep5_filtered.iloc[0])
sweep5_filtered = sweep5_filtered[1:]
print sweep5_filtered.shape
sweep5_filtered.head()

(11469, 400)


,ncdsid serial number,CMI:83 F34 Any accident/assault since March 1981,CMI:83 F35 No. of accidents/assaults since March 1981,CMI:83 F36b) ACCIDENT/ASSAULT 1: Age occurred,CMI:83 F36c) ACCIDENT/ASSAULT 1: Incident type,CMI:83 F36d) ACCIDENT/ASSAULT 1: Admitted/outpatient,CMI:83 F36b) ACCIDENT/ASSAULT 2: Age occurred,CMI:83 F36c) ACCIDENT/ASSAULT 2: Incident type,CMI:83 F36d) ACCIDENT/ASSAULT 2: Admitted/outpatient,CMI:83 F36b) ACCIDENT/ASSAULT 3: Age occurred,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
1,N10001N,2,,,,,,,,,...,,,1,2,,,,,,
2,N10002P,2,,,,,,,,,...,,,1,2,,,,,,
3,N10004R,2,,,,,,,,,...,,,1,2,,,,,,
4,N10007U,2,,,,,,,,,...,,,1,2,,,,,,
5,N10009W,2,,,,,,,,,...,,,1,2,,,,,,


In [46]:
def check_nan(df, name):
    
    print len(df[name][df[name].isnull()])

In [28]:
sweep5_filtered.replace(' ',np.nan,inplace=True)

In [30]:
sweep5_filtered.head(5)

,ncdsid serial number,CMI:83 F34 Any accident/assault since March 1981,CMI:83 F35 No. of accidents/assaults since March 1981,CMI:83 F36b) ACCIDENT/ASSAULT 1: Age occurred,CMI:83 F36c) ACCIDENT/ASSAULT 1: Incident type,CMI:83 F36d) ACCIDENT/ASSAULT 1: Admitted/outpatient,CMI:83 F36b) ACCIDENT/ASSAULT 2: Age occurred,CMI:83 F36c) ACCIDENT/ASSAULT 2: Incident type,CMI:83 F36d) ACCIDENT/ASSAULT 2: Admitted/outpatient,CMI:83 F36b) ACCIDENT/ASSAULT 3: Age occurred,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
1,N10001N,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,N10002P,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
3,N10004R,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
4,N10007U,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
5,N10009W,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN


Merge them!!!

In [38]:
merged = pd.merge(sweep0_3_filtered,sweep5_filtered,on = "ncdsid serial number")
print merged.shape
merged.head()

(11469, 1476)


,ncdsid serial number,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,N10001N,-1,1,3,1,3,5,1,2,2,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
1,N10002P,-1,3,2,1,2,4,2,2,2,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,N10004R,-1,3,3,1,2,4,2,2,2,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
3,N10007U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
4,N10009W,-1,12,2,2,2,4,1,2,2,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN


# Time to do something on the merged data!!

In [20]:
a = pd.DataFrame({"index":[1,2,"3.5",4,5,6,7],"a":[2,2,3,4,5,6,7]})
a["index"] =  pd.to_numeric(a.iloc[:,1],errors='coerce')
a

,a,index
0,2,1.0
1,2,2.0
2,3,3.5
3,4,4.0
4,5,5.0
5,6,6.0
6,7,7.0


In [39]:
merged.drop("ncdsid serial number",1,inplace = True) 
print merged.shape
merged.head()

(11469, 1475)


,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,1PD Parental sit of illegitimate kids,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,-1,1,3,1,3,5,1,2,2,-1,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,3,2,1,2,4,2,2,2,-1,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,-1,3,3,1,2,4,2,2,2,-1,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN
4,-1,12,2,2,2,4,1,2,2,-1,...,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN


Now, convert all the data to numeric (now they are all object) and replace negetive values with NaN as well!

In [40]:
#convert all the predictors to numeric
def to_numeric(df):
    for index,column in enumerate(df.columns):
        temp = pd.to_numeric(df.iloc[:,index],errors='coerce')
        temp[temp < 0] = np.NaN
        df[column] = temp
    return df

merged = to_numeric(merged)
merged["CMI:75 F3a) Epilepsy: Ever suffered"].unique()

array([  3.,   1.,  nan,   2.])

In [41]:

print "number of nan in reponse: ",check_nan(merged,"CMI:75 F3a) Epilepsy: Ever suffered")

number of nan in reponse:  143


In [69]:
#delete rows with nan in response varialbe!!!!! Important!
merged_complete = merged.dropna(subset = ["CMI:75 F3a) Epilepsy: Ever suffered"])

In [70]:
print merged_complete.shape
merged_complete.head(10)

(11326, 1475)


,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,1PD Parental sit of illegitimate kids,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,NaN,1.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,3.0,1.0,2.0,4.0,2.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,12.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,0.0,1.0,1.0,8.0,NaN,6.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,0.0,1.0,1.0,4.0,6.0,3.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,0.0,1.0,1.0,6.0,8.0,4.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,2.0,2.0,1.0,2.0,4.0,1.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1.0,3.0,1.0,4.0,6.0,1.0,2.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mid = df['Mid']
df.drop(labels=['Mid'], axis=1,inplace = True)
df.insert(0, 'Mid', mid)

In [80]:

target = "CMI:75 F3a) Epilepsy: Ever suffered"
move = merged_complete[target]
temp = merged_complete.drop(labels=[target], axis=1)
temp.insert(0, target, move)
temp.head()

,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,3.0,NaN,1.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,3.0,3.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,12.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
temp2 = temp[temp[target] != 2]
print temp2.shape
temp2.head()

(11304, 1475)


,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,3.0,NaN,1.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,3.0,3.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,12.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
temp.to_csv("clean_data.csv",sep=",",index_label=False)

In [62]:
#move the response to last place
target = "CMI:75 F3a) Epilepsy: Ever suffered"
cols = merged_complete.columns.tolist()
cols.insert(0, cols.pop(cols.index(target)))
data = merged_complete[cols]
data.head(5)

,CMI:75 F3a) Epilepsy: Ever suffered,1P Child 8 years-old at NCDS1 Parental,1P Family moves since child's birth,1P Moves out of local area,1P Informant's relationship to child,"1P No. kids undr 21 hhld,inc liv away",1P Number of people in the household,"1P Child's posn hhld ,inc those away",1P Relationship person acting as mother,1P Relationship person acting as father,...,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #5,CMI:94 F74d) ROAD TRAFFIC ACCIDENT 4: Offence #6,CMI:95 F75a) CHECK: F64 - Has driving licence,CMI:95 F75b) Ever been convicted of traffic offence (excl F73-74),CMI:95 F75c) Additional traffic offence conviction #1,CMI:95 F75c) Additional traffic offence conviction #2,CMI:95 F75c) Additional traffic offence conviction #3,CMI:95 F75c) Additional traffic offence conviction #4,CMI:95 F75c) Additional traffic offence conviction #5,CMI:95 F75c) Additional traffic offence conviction #6
0,3.0,NaN,1.0,3.0,1.0,3.0,5.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,3.0,2.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,3.0,3.0,1.0,2.0,4.0,2.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,12.0,2.0,2.0,2.0,4.0,1.0,2.0,2.0,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
for name in merged_complete.columns:
    check_nan(merged_complete,name)

10987
1527
1437
1359
1417
1752
1411
1367
1369
11020
1425
1449
1430
1417
1551
1445
1406
1412
1406
1412
1411
1408
1409
1411
1411
1406
1418
1416
1412
1415
1415
1417
1413
1416
1414
1374
11148
11149
11148
11148
1411
1420
1417
1416
1416
1416
1418
1416
1419
1419
1422
1425
1430
1418
1417
1426
1336
1567
1423
1438
1424
1432
1427
1423
1430
1444
1435
1465
10556
11109
11216
9758
3187
1453
1534
1483
971
1426
971
1437
1439
1459
1434
1442
1491
1424
1426
1428
1411
1455
1401
1402
1412
1407
1459
1482
1424
1438
1479
1678
1407
1404
1405
1415
1428
1420
1422
1427
1421
1419
1428
1443
1477
1438
1417
1433
1430
1450
1425
1440
1451
1472
1454
1441
1440
1438
1441
1438
1440
1441
1440
1448
1449
1444
1444
1444
1450
1447
1446
1443
1445
1464
1438
1439
1441
1444
1439
1448
1445
1444
1431
1453
1433
1445
1435
1424
1430
1431
1441
1440
1442
1441
1439
1459
1451
1424
1426
1440
1430
1424
1432
1429
1428
1430
1433
1432
1429
1427
1521
1428
1433
1437
1432
1431
1428
1428
1429
1431
1432
1435
1434
1479
1416
1429
11212
1802
1465
1500
17

In [153]:
#if df[column].unique()<15, dummify it
def get_dummy(df,stand = False):
    
    #copy dataframes
    data = df.copy()
    
    #Create a new data frame 
    x = pd.DataFrame({}) 
    predictor_list = []

    # Iterate over variables
    for i,column in enumerate(data.columns.values):
        #delete some errors
        if data[column].shape == (11326,):
            if (len(data[column].unique()) < 15) and column != "CMI:75 F3a) Epilepsy: Ever suffered":
                #dummify categorical predictors
                encoding = pd.get_dummies(data[column])

                #get dummified predictors column names and store them
                temp = []
                names = encoding.columns.values
                for i in range(len(names)):
                    temp.append(column + "_" + str(names[i]))


                # append expanded attribute to data frame
                encoding.columns = temp
                x = pd.concat([x, encoding], axis=1)

            else:
                #if contrain nan, replace with mean in the data
                data[column].fillna(data[column].mean(), inplace=True)

                #standardize 
                if stand:
                    mean = data[column].mean()
                    std = data[column].std()
                    data[column] = (data[column] - mean)/std

                x = pd.concat([x, data[column]], axis=1)
        else:  print "error"    
            
            
    #if predictors contain info related to epilepsy, delete it
    for name in x.columns.values:
        if "epilep" in name and name != "CMI:75 F3a) Epilepsy: Ever suffered":
            x.drop(name, axis=1, inplace=True)
    
    predictor_list = x.columns.values 

    print x.shape
    print "number of variables: ",len(predictor_list)
    return x

In [154]:
merged_final = get_dummy(merged_complete,stand=True)

error
error
error
error
error
error
error
error
error
error
error
error
(11326, 6234)
number of variables:  6234


In [155]:
merged_final.head(5)

,1P Child 8 years-old at NCDS1 Parental_1.0,1P Family moves since child's birth,1P Moves out of local area_1.0,1P Moves out of local area_2.0,1P Moves out of local area_3.0,1P Informant's relationship to child_1.0,1P Informant's relationship to child_2.0,1P Informant's relationship to child_3.0,1P Informant's relationship to child_4.0,"1P No. kids undr 21 hhld,inc liv away",...,CMI:95 F75c) Additional traffic offence conviction #3_6.0,CMI:95 F75c) Additional traffic offence conviction #4_2.0,CMI:95 F75c) Additional traffic offence conviction #4_4.0,CMI:95 F75c) Additional traffic offence conviction #4_5.0,CMI:95 F75c) Additional traffic offence conviction #4_6.0,CMI:95 F75c) Additional traffic offence conviction #5_1.0,CMI:95 F75c) Additional traffic offence conviction #5_5.0,CMI:95 F75c) Additional traffic offence conviction #5_6.0,CMI:95 F75c) Additional traffic offence conviction #6_2.0,CMI:95 F75c) Additional traffic offence conviction #6_6.0
0,0.0,-0.104884,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.020743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.276515,0.0,1.0,0.0,1.0,0.0,0.0,0.0,-0.659082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.276515,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.659082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,7.492811,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-0.659082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Cleaned data

### 1. The next step is to maybe dummify the categorical predictors. May be check df[column].unique()

The value of a categorical predictors can be negtive, e.g.

>Pos. = 68	Variable = n97	Variable label = 1P Moves out of local area

>This variable is  numeric, the SPSS measurement level is ORDINAL

>SPSS user missing values = -1.0 thru None

	>Value label information for n97
    
	>Value = 1.0	Label = DK,Inapplicable
    
	>Value = 2.0	Label = Yes
    
	>Value = 3.0	Label = No

	>**Value = -1.0  Label = NA**
    
**Notice the last line, values = -1, in the dataset, I have already converted all the negtive values to nan.** 

- if >15 (I have seen the discriptions and find this general rule, but there are exceptions, you are free to change the threshold), numeric; if < 15, dummify. May be delete NaN columns after dummifying. Or fill in the most frequent category.

### 2. There are several things coming to me mind:

- If doing logistic regression with L2/L1 penalty, you can both try standardize it or not. It would be better if you standardize it (may be use scikit learn processing.standardize(with_mean = True,with_std = True)). Then find the predictors with highest postive coefficients and corresponding to the variable names!  
- If do decision trees to find important variable, try visualize the top variables!

In [23]:
#predictors 
x = merged.drop("CMI:75 F3a) Epilepsy: Ever suffered",1,inplace = False).values

#labels for all predictors
labels = merged.drop("CMI:75 F3a) Epilepsy: Ever suffered",1,inplace = False).columns

#classification 
y = merged["CMI:75 F3a) Epilepsy: Ever suffered"].values
y[y != 1] = 0

In [24]:
print "Number of epilepsy patients: ", y.sum()

Number of epilepsy patients:  141.0


In [28]:
# x = merged.drop("CMI:75 F3a) Epilepsy: Ever suffered",1,inplace = False)
# x.to_csv('x_cleaned_data.csv')
# np.savetxt("y_cleaned_data.csv", y, delimiter=",")